In [18]:
import torch
import torch.nn as nn
import re

from torch.nn import functional as F

with open('../src/summaries.txt', 'r', encoding='utf8') as f:
    text = f.read()

# ensure that there are only Latin and special character wording. CJK characters are removed
pattern = re.compile(r'[^\x00-\x7F0-9\[\]]+')
text = pattern.sub('', text)

# find all unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

enc_map = {}
dec_map = {}

for i, character in enumerate(chars):
    enc_map[character] = i
    dec_map[i] = character

def encode(s : str, enc_map : dict) -> list:
    ls = []
    for char in s:
        ls.append(enc_map[char])
    return ls

def decode(ls : list, dec_map : dict) -> str:
    char_list = []
    ls = ls.tolist()
    for i in ls: 
        char_list.append(dec_map[i]) 
    s = ''.join(char_list)
    return s

data = torch.tensor(encode(text, enc_map), dtype=torch.long)

n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

print(decode(train_data[:10], dec_map))

Title: Kan
